<a href="https://colab.research.google.com/github/jdomdev/ai-small-language-model/blob/feature%2Fimdb-sentiment/train_sentiment_colab_full_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Última versión del Notebook con primer entrenamiento realizado, pero no guardado de modelo

El modelo actual pesa 257MB, lo que confirma que la división será necesaria para subirlo a GitHub.

Vamos a crear el notebook de Colab paso a paso, conectarlo con el repositorio de GitHub propio (**jdomdev/ai-small-language-model**) y entrenar el modelo DistilBERT en la nube de Google(Colab) con todos los registros los datasets (train: 25.000 / test: 25.000)

Contenido del Notebook de Colab (train_sentiment_colab_full_data.ipynb)

  **Error en TrainingArguments()**


  El mensaje de error nos da la pista clave:
  sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.38.2 which is incompatible.

  Esto significa que tu Colab tiene sentence-transformers instalado, y esta librería necesita una versión de transformers que sea
  4.41.0 o más nueva. La que yo te sugerí (4.38.2) es demasiado vieja para tu entorno actual.

  La Solución Definitiva

  Vamos a corregirlo. En lugar de forzar una versión específica, simplemente pediremos la última versión estable. pip es lo
  suficientemente inteligente como para encontrar un conjunto de versiones que funcionen entre sí.

  Reemplaza la celda de instalación que te di antes por esta nueva celda:

    1 # Paso 1: Instalar las versiones más recientes y compatibles de las librerías
    2 # Usamos --upgrade para asegurarnos de que tenemos las últimas versiones que no entren en conflicto.
    3 print("Instalando las últimas versiones de transformers, datasets y accelerate...")
    4 !pip install --upgrade -q transformers datasets accelerate
    5
    6 # Paso 2: Forzar el reinicio del entorno para que Colab use las nuevas librerías
    7 # La sesión se reiniciará. Esto es normal y esperado.
    8 import os
    9 print("Instalación completa. Reiniciando el entorno de ejecución...")
   10 os.kill(os.getpid(), 9)

  ¿Qué hace este nuevo código?

   1. `!pip install --upgrade -q transformers datasets accelerate`:
       * --upgrade: Le dice a pip que instale la versión más reciente disponible que sea compatible con el resto del entorno.
       * Ya no fijamos versiones (==X.Y.Z), lo que le da a pip la libertad de resolver los conflictos que vimos antes.
       * -q: Es solo para que la salida de la instalación sea más limpia (modo "quiet").

   2. `os.kill(...)`: Mantenemos este comando porque es la forma más segura de garantizar que el entorno se reinicia y carga las
      librerías que acabamos de instalar.

  Pasos a seguir

   1. Elimina o reemplaza la celda de instalación anterior con el nuevo código que te he puesto arriba.
   2. Ejecuta esta nueva celda.
   3. Verás los mensajes de instalación y luego el aviso de que la sesión se ha reiniciado ("session crashed"). Esto es correcto.
   4. Ahora, ejecuta el resto de las celdas de tu notebook en orden.

  Ahora, al ejecutar el resto de tu código, el TypeError original sobre evaluation_strategy debería estar resuelto
  definitivamente, ya que tendrás una versión de transformers reciente y compatible con todo tu entorno de Colab.

In [1]:
# Paso 1: Instalar las versiones más recientes y compatibles de las librerías
# Usamos --upgrade para asegurarnos de que tenemos las últimas versiones que no entren en conflicto.
"""
print("Instalando las últimas versiones de transformers, datasets y accelerate...")
!pip install --upgrade -q transformers datasets accelerate

# Paso 2: Forzar el reinicio del entorno para que Colab use las nuevas librerías
# La sesión se reiniciará. Esto es normal y esperado.
import os
print("Instalación completa. Reiniciando el entorno de ejecución...")
os.kill(os.getpid(), 9)
"""

'\nprint("Instalando las últimas versiones de transformers, datasets y accelerate...")\n!pip install --upgrade -q transformers datasets accelerate\n \n# Paso 2: Forzar el reinicio del entorno para que Colab use las nuevas librerías\n# La sesión se reiniciará. Esto es normal y esperado.\nimport os\nprint("Instalación completa. Reiniciando el entorno de ejecución...")\nos.kill(os.getpid(), 9)\n'

In [2]:
#!pip install --upgrade transformers datasets

#**Bloque 1: Configuración Inicial e Importaciones**

Este bloque se encarga de instalar las librerías necesarias (aunque muchas ya están en Colab, es buena práctica incluirlas para
asegurar la reproducibilidad), montar Google Drive y definir las rutas.

##**@title 1. Configuración Inicial e Importaciones**

In [3]:
# Instalar o actualizar librerías necesarias (muchas ya están en Colab, pero es buena práctica)
#!pip install --upgrade transformers[torch] datasets evaluate scikit-learn accelerate pandas numpy

In [4]:
# Importar librerías
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset
from evaluate import load
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from sklearn.model_selection import train_test_split
import re
import os

In [5]:
# Montar Google Drive para guardar/cargar archivos grandes
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Definir la ruta base en Google Drive para guardar los resultados y el modelo
# Asegúrate de que esta carpeta exista en tu Google Drive
DRIVE_BASE_PATH = "/content/drive/My Drive/Colab Notebooks/mod03-projs/ai-small-language-model/"
os.makedirs(DRIVE_BASE_PATH, exist_ok=True)

In [7]:
print("Configuración inicial completada.")

Configuración inicial completada.


#**Bloque 2: Carga y Guardado de Datasets Originales a CSV**

Aquí cargaremos los datasets completos de IMDb y los guardaremos como CSVs separados en tu Google Drive.

**@title 2. Carga y Guardado de Datasets Originales a CSV**

Cargamos el Hugging Face Token(HF_Token)

In [8]:
from google.colab import userdata
import os

# Intenta obtener el token de Hugging Face del administrador de secretos
HF_TOKEN = userdata.get('HF_TOKEN')

# Opcional: Configura la variable de entorno HF_HOME si necesitas especificar una ubicación de caché diferente
# os.environ['HF_HOME'] = '/content/drive/MyDrive/hf_cache'

# Esto configurará el token para las bibliotecas de Hugging Face automáticamente
if HF_TOKEN:
    os.environ['HF_TOKEN'] = HF_TOKEN
    print("Token de Hugging Face cargado desde secretos de Colab.")
else:
    print("Advertencia: HF_TOKEN no encontrado en secretos de Colab.")


Token de Hugging Face cargado desde secretos de Colab.


Sólo actualizar en caso de que la carga del dataset de problemas:

"These are the names of the Python packages to be installed or upgraded. datasets is the Hugging Face library for easily accessing and working with datasets, and fsspec is a library that provides a file-system-like interface to various storage backends, which datasets uses internally."

In [9]:
# !pip install --upgrade datasets fsspec

In [10]:
print("Cargando el dataset IMDb completo...")
dataset = load_dataset("imdb")
print("Dataset IMDb cargado.")

Cargando el dataset IMDb completo...


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset IMDb cargado.


In [11]:
# Convertir a Pandas DataFrame y guardar como CSV
df_train_original = pd.DataFrame(dataset["train"])
df_test_original = pd.DataFrame(dataset["test"])

In [12]:
train_csv_path = os.path.join(DRIVE_BASE_PATH, "imdb_train_original.csv")
test_csv_path = os.path.join(DRIVE_BASE_PATH, "imdb_test_original.csv")

In [13]:
df_train_original.to_csv(train_csv_path, index=False)
df_test_original.to_csv(test_csv_path, index=False)

In [14]:
print(f"Dataset de entrenamiento original guardado en: {train_csv_path}")
print(f"Dataset de prueba original guardado en: {test_csv_path}")
print(f"Tamaño del dataset de entrenamiento original: {len(df_train_original)} registros")
print(f"Tamaño del dataset de prueba original: {len(df_test_original)} registros")

Dataset de entrenamiento original guardado en: /content/drive/My Drive/Colab Notebooks/mod03-projs/ai-small-language-model/imdb_train_original.csv
Dataset de prueba original guardado en: /content/drive/My Drive/Colab Notebooks/mod03-projs/ai-small-language-model/imdb_test_original.csv
Tamaño del dataset de entrenamiento original: 25000 registros
Tamaño del dataset de prueba original: 25000 registros


#**Bloque 3: Unificación y Limpieza de Datos**

Este es el bloque crucial para la limpieza y unificación. Incluiré una función de limpieza básica y comprobaciones de
nulos/duplicados.

**@title 3. Unificación y Limpieza de Datos**

In [15]:
print("Unificando datasets de entrenamiento y prueba...")
df_full = pd.concat([df_train_original, df_test_original], ignore_index=True)
print(f"Dataset unificado creado con {len(df_full)} registros.")

Unificando datasets de entrenamiento y prueba...
Dataset unificado creado con 50000 registros.


In [16]:
print("Realizando comprobaciones de limpieza de datos...")

Realizando comprobaciones de limpieza de datos...


In [17]:
# Comprobar nulos
print(f"Valores nulos por columna antes de la limpieza:\n{df_full.isnull().sum()}")

Valores nulos por columna antes de la limpieza:
text     0
label    0
dtype: int64


In [18]:
# Comprobar duplicados
print(f"Registros duplicados antes de la limpieza: {df_full.duplicated().sum()}")
if df_full.duplicated().sum() > 0:
    df_full.drop_duplicates(inplace=True)
    print(f"Registros duplicados eliminados. Nuevo tamaño: {len(df_full)}")

Registros duplicados antes de la limpieza: 418
Registros duplicados eliminados. Nuevo tamaño: 49582


In [19]:
# Función de limpieza de texto
def clean_text(text):
    text = str(text).lower() # Convertir a string y a minúsculas
    text = re.sub(r'<br />', ' ', text) # Eliminar etiquetas <br />
    text = re.sub(r'[^a-z0-9\s]', '', text) # Eliminar caracteres especiales (mantener letras, números, espacios)
    text = re.sub(r'\s+', ' ', text).strip() # Eliminar espacios extra
    return text

In [20]:
print("Aplicando limpieza de texto a la columna 'text'...")
df_full['text'] = df_full['text'].apply(clean_text)

Aplicando limpieza de texto a la columna 'text'...


In [21]:
# Comprobar si hay "nulos" como cadenas vacías o solo espacios después de la limpieza
print(f"Registros con texto vacío después de la limpieza: {(df_full['text'] == '').sum()}")
if (df_full['text'] == '').sum() > 0:
    df_full = df_full[df_full['text'] != '']
    print(f"Registros con texto vacío eliminados. Nuevo tamaño: {len(df_full)}")

Registros con texto vacío después de la limpieza: 0


In [22]:
# Guardar el dataset unificado y limpio
full_cleaned_csv_path = os.path.join(DRIVE_BASE_PATH, "imdb_full_cleaned.csv")
df_full.to_csv(full_cleaned_csv_path, index=False)
print(f"Dataset unificado y limpio guardado en: {full_cleaned_csv_path}")

Dataset unificado y limpio guardado en: /content/drive/My Drive/Colab Notebooks/mod03-projs/ai-small-language-model/imdb_full_cleaned.csv


#**Bloque 4: División 80/20 para Entrenamiento y Prueba**

Aquí dividiremos el dataset limpio en 80% para entrenamiento y 20% para prueba.

**@title 4. División 80/20 para Entrenamiento y Prueba**

In [23]:
print("Dividiendo el dataset unificado en 80% entrenamiento y 20% prueba...")
train_df, test_df = train_test_split(df_full, test_size=0.2, random_state=42, stratify=df_full['label'])

Dividiendo el dataset unificado en 80% entrenamiento y 20% prueba...


In [24]:
# Convertir DataFrames de Pandas a objetos Dataset de Hugging Face
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [25]:
# Eliminar la columna '__index_level_0__' que se añade automáticamente al convertir de pandas
train_dataset = train_dataset.remove_columns(["__index_level_0__"])
test_dataset = test_dataset.remove_columns(["__index_level_0__"])

In [26]:
print(f"Tamaño del dataset de entrenamiento (80%): {len(train_dataset)} registros")
print(f"Tamaño del dataset de prueba (20%): {len(test_dataset)} registros")

Tamaño del dataset de entrenamiento (80%): 39665 registros
Tamaño del dataset de prueba (20%): 9917 registros


#**Bloque 5: Tokenización y Carga del Modelo (Adaptado del Script Original)**

Este bloque es una adaptación directa del train_sentiment_model.py.

**@title 5. Tokenización y Carga del Modelo**

In [27]:
# Cargar el Tokenizador
print("\nCargando el tokenizador DistilBERT...")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


Cargando el tokenizador DistilBERT...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [28]:
# Función de Preprocesamiento
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

In [29]:
print("\nPreprocesando el dataset de entrenamiento y prueba...")
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)


Preprocesando el dataset de entrenamiento y prueba...


Map:   0%|          | 0/39665 [00:00<?, ? examples/s]

Map:   0%|          | 0/9917 [00:00<?, ? examples/s]

In [30]:
# Cargar el Modelo
print("\nCargando el modelo DistilBERT para clasificación de secuencias...")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Cargando el modelo DistilBERT para clasificación de secuencias...


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#**Bloque 6: Definición de Métricas y Configuración de Entrenamiento**

También adaptado de tu script original.

**@title 6. Definición de Métricas y Configuración de Entrenamiento**

In [31]:
# Definir Métricas de Evaluación
print("\nDefiniendo métricas de evaluación...")
metric = load("accuracy")
f1_metric = load("f1")
precision_metric = load("precision")
recall_metric = load("recall")


Definiendo métricas de evaluación...


In [32]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    precision = precision_metric.compute(predictions=predictions, references=labels, average="weighted")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="weighted")
    return {**accuracy, **f1, **precision, **recall}

In [33]:
import transformers
print(f"La versión de Transformers que se está usando es: {transformers.__version__}")

La versión de Transformers que se está usando es: 4.54.0


In [34]:
# Configurar Argumentos de Entrenamiento
"""
print("\nConfigurando argumentos de entrenamiento...")
training_args = TrainingArguments(
    output_dir=os.path.join(DRIVE_BASE_PATH, "results"), # Directorio para guardar los resultados en Drive
    num_train_epochs=3,                   # Número de épocas de entrenamiento
    per_device_train_batch_size=16,  # Tamaño del batch por dispositivo (GPU/CPU)
    per_device_eval_batch_size=16,   # Tamaño del batch para evaluación
    warmup_steps=500,                   # Número de pasos para el calentamiento del learning rate
    weight_decay=0.01,                  # Regularización L2
    logging_dir=os.path.join(DRIVE_BASE_PATH, "logs"), # Directorio para los logs de TensorBoard en Drive
    logging_steps=100,
    report_to="none",                   # No reportar a ninguna plataforma (ej. wandb)
    save_strategy="epoch",            # Guardar el modelo al final de cada época
    evaluation_strategy="epoch",      # <<-- Añadido: Evaluar al final de cada época para que load_best_model_at_end funcione
    load_best_model_at_end=True,     # Cargar el mejor modelo al final del entrenamiento
    metric_for_best_model="f1",      # Métrica para determinar el mejor modelo
)
"""

'\nprint("\nConfigurando argumentos de entrenamiento...")\ntraining_args = TrainingArguments(\n    output_dir=os.path.join(DRIVE_BASE_PATH, "results"), # Directorio para guardar los resultados en Drive\n    num_train_epochs=3,                   # Número de épocas de entrenamiento\n    per_device_train_batch_size=16,  # Tamaño del batch por dispositivo (GPU/CPU)\n    per_device_eval_batch_size=16,   # Tamaño del batch para evaluación\n    warmup_steps=500,                   # Número de pasos para el calentamiento del learning rate\n    weight_decay=0.01,                  # Regularización L2\n    logging_dir=os.path.join(DRIVE_BASE_PATH, "logs"), # Directorio para los logs de TensorBoard en Drive\n    logging_steps=100,\n    report_to="none",                   # No reportar a ninguna plataforma (ej. wandb)\n    save_strategy="epoch",            # Guardar el modelo al final de cada época\n    evaluation_strategy="epoch",      # <<-- Añadido: Evaluar al final de cada época para que load_be

Bloque 7: Entrenamiento del Modelo

@title 7. Entrenamiento del Modelo

In [37]:
import os
import numpy as np
# Importaciones necesarias para las métricas
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# Importaciones de Hugging Face Transformers y Datasets
# Estas deberían estar ya en tu entorno Colab o haber sido instaladas previamente.
# Asumo que `Trainer`, `TrainingArguments`, `AutoModelForSequenceClassification`
# y `tokenized_datasets` (que contiene "train" y "test" datasets) están disponibles
# del contexto del notebook previo.
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification

# ==============================================================================
# 1. FUNCIÓN PARA CALCULAR MÉTRICAS
# El Trainer necesita una función que le diga CÓMO calcular las métricas
# en el conjunto de evaluación durante el entrenamiento. Definimos esta función
# para que reporte las métricas clave para problemas de clasificación binaria:
# 'f1' (F1-score), 'precision', 'recall' y 'accuracy' (exactitud).
# ==============================================================================
def compute_metrics(pred):
    """
    Calcula y devuelve un diccionario de métricas de evaluación a partir de las predicciones del modelo.

    Args:
        pred (EvalPrediction): Un objeto que contiene los logits (predicciones crudas)
                               y las etiquetas verdaderas del conjunto de evaluación.

    Returns:
        dict: Un diccionario con las métricas calculadas (accuracy, f1, precision, recall).
    """
    # `pred.label_ids` contiene las etiquetas verdaderas del conjunto de evaluación.
    labels = pred.label_ids
    # `pred.predictions` contiene los logits (salidas crudas del modelo antes de la activación final).
    # `np.argmax(pred.predictions, axis=1)` convierte estos logits en las clases predichas
    # (0 o 1 para clasificación binaria) seleccionando el índice de la mayor probabilidad.
    preds = np.argmax(pred.predictions, axis=1)

    # `precision_recall_fscore_support` calcula la precisión, el recall y el F1-score.
    # `average='binary'` es crucial para problemas de clasificación binaria, donde se calcula
    # para la clase positiva (generalmente 1).
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    # `accuracy_score` calcula la proporción de predicciones correctas.
    acc = accuracy_score(labels, preds)

    # Se devuelve un diccionario con todas las métricas calculadas. El Trainer usará esto
    # para registrar el progreso de la evaluación.
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# ==============================================================================
# 2. CONFIGURAR ARGUMENTOS DE ENTRENAMIENTO (TrainingArguments)
# Esta sección define los parámetros y estrategias para el proceso de entrenamiento.
# Usaremos `eval_steps` y `save_steps` para evaluar y guardar checkpoints
# de manera periódica, específicamente al final de cada época completa.
# ==============================================================================
print("\nConfigurando argumentos de entrenamiento...")

# Calculamos cuántos pasos de entrenamiento hay en una época.
# Este cálculo es crucial para la estrategia de evaluación y guardado.
# Si el dataset IMDb tiene 25,000 ejemplos de entrenamiento y el tamaño del batch es 16,
# entonces los pasos por época serán 25000 / 16. Usamos la división entera (`//`)
# para obtener un número de pasos completo.
# Este valor (`steps_per_epoch`) se usará para configurar `eval_steps` y `save_steps`.
# Esto significa que el modelo se evaluará y se guardará un checkpoint *cada vez*
# que se haya procesado el dataset de entrenamiento completo una vez.
steps_per_epoch = 25000 // 16 # Asumiendo 25,000 ejemplos de entrenamiento en el dataset IMDb.

# Se instancia la clase `TrainingArguments` con los parámetros deseados.
training_args = TrainingArguments(
    # --- ¡AQUÍ ESTÁ EL CAMBIO CLAVE! ---
    # Usamos os.path.join para construir la ruta completa dentro de tu Drive.
    output_dir=os.path.join(DRIVE_BASE_PATH, "results"), # Guardará los checkpoints en Drive, # Directorio donde el Trainer guardará los checkpoints del modelo,
                            # los logs y otros resultados del entrenamiento.
    num_train_epochs=3,     # Número total de épocas (pasos completos sobre el dataset de entrenamiento)
                            # para realizar el entrenamiento.
    per_device_train_batch_size=16, # Tamaño del batch de datos por dispositivo (GPU/CPU) para el entrenamiento.
                                    # Un tamaño de batch adecuado puede optimizar el uso de memoria y la velocidad.
    per_device_eval_batch_size=16,  # Tamaño del batch de datos por dispositivo para la evaluación.
                                    # Puede ser diferente al tamaño del batch de entrenamiento.
    warmup_steps=500,       # Número de pasos para el "calentamiento" del learning rate. Durante estos pasos,
                            # el learning rate aumenta gradualmente desde 0 hasta el valor inicial,
                            # lo que puede ayudar a estabilizar el entrenamiento.
    weight_decay=0.01,      # Factor de regularización L2 (descomposición de peso). Ayuda a prevenir el sobreajuste
                            # penalizando los pesos grandes del modelo.
    logging_dir=os.path.join(DRIVE_BASE_PATH, "logs"), # Guardará los logs de TensorBoard en Drive   # Directorio donde se guardarán los logs de TensorBoard para visualizar el progreso del entrenamiento.
    logging_steps=100,      # Frecuencia (en pasos de entrenamiento) con la que se registrarán las métricas y el progreso.
    report_to="none",       # Deshabilita la integración con herramientas de reporte externas como Weights & Biases (wandb).
                            # Puedes cambiar esto si deseas integrar con alguna de ellas.

    # --- ¡AQUÍ ESTÁ LA CLAVE PARA LA EVALUACIÓN Y GUARDADO PERIÓDICO! ---
    # `eval_steps`: Frecuencia (en pasos de entrenamiento) con la que se realizará una evaluación
    # en el `eval_dataset`. Al establecerlo a `steps_per_epoch`, evaluamos al final de cada época.
    eval_steps=steps_per_epoch,

    # `save_steps`: Frecuencia (en pasos de entrenamiento) con la que se guardará un checkpoint
    # del modelo. Al establecerlo a `steps_per_epoch`, guardamos un checkpoint al final de cada época.
    save_steps=steps_per_epoch,

    # IMPORTANTE: `load_best_model_at_end` no se puede usar con `eval_steps` y `save_steps`
    # de esta manera en algunas versiones de `Trainer` (o si se requiere una lógica de selección
    # de modelo más específica). Por eso, se realizará la carga del mejor modelo manualmente
    # después del entrenamiento, basándonos en los resultados de evaluación registrados.
    # load_best_model_at_end=True, # Deshabilitado o comentado por la estrategia manual
    # metric_for_best_model="f1",  # Deshabilitado o comentado por la estrategia manual
)

# ==============================================================================
# 3. CREAR Y ENTRENAR EL TRAINER
# Esta sección instancia el `Trainer` con el modelo, los argumentos de entrenamiento,
# y los conjuntos de datos, y luego inicia el proceso de entrenamiento.
# ==============================================================================
print("\nCreando el Trainer...")
# Se crea el objeto `Trainer`, que encapsula toda la lógica de entrenamiento.
trainer = Trainer(
    model=model,  # El modelo de Transformers (ej. DistilBERT) a entrenar.
    args=training_args,  # Los argumentos de entrenamiento definidos en la sección anterior.
    # Conjunto de datos de entrenamiento. Asegúrate de que `tokenized_datasets`
    # esté definido y contenga las claves "train" y "test" (o los nombres que uses para tus splits).
    train_dataset=tokenized_train_dataset,
    # Conjunto de datos de evaluación. Este dataset se usará para medir el rendimiento del modelo
    # en datos no vistos durante el entrenamiento, según la frecuencia definida por `eval_steps`.
    eval_dataset=tokenized_test_dataset,
    # Se especifica la función `compute_metrics` que se utilizará para calcular y reportar
    # las métricas durante las evaluaciones periódicas.
    compute_metrics=compute_metrics,
    # Se pasa el tokenizador al Trainer. Esto es una buena práctica ya que el Trainer
    # puede guardar el tokenizador junto con el modelo, facilitando la carga posterior
    # del modelo entrenado con su tokenizador correspondiente.
    tokenizer=tokenizer,
)

print("\nIniciando el entrenamiento del modelo...")
# Inicia el ciclo de entrenamiento. El `Trainer` gestionará las épocas, los pasos,
# las evaluaciones y el guardado de checkpoints según los `training_args`.
trainer.train()
print("\nEntrenamiento completado.")


# ==============================================================================
# 4. ENCONTRAR Y CARGAR EL MEJOR MODELO MANUALMENTE
# Dado que `load_best_model_at_end` no se usó (o no era compatible con la estrategia
# de evaluación/guardado deseada), necesitamos inspeccionar el historial de entrenamiento
# para identificar el checkpoint que produjo el mejor rendimiento (basado en 'f1')
# y luego cargar ese modelo específico.
# ==============================================================================
print("\nBuscando el mejor checkpoint según la métrica 'f1'...")

# Inicializamos las variables para rastrear el F1-score más alto encontrado
# y la ruta al directorio de ese checkpoint.
best_metric = -1          # Se inicializa con un valor muy bajo para asegurar que la primera métrica real sea mejor.
best_checkpoint_dir = None # Se inicializa a None; se actualizará con la ruta del mejor checkpoint.

# El historial completo de logs de entrenamiento y evaluación está disponible
# en `trainer.state.log_history`. Este es una lista de diccionarios, donde cada diccionario
# representa un evento de log (ej., progreso del entrenamiento, resultados de evaluación).
for log in trainer.state.log_history:
    # Filtramos las entradas del log que corresponden a una evaluación, identificadas
    # por la presencia de la clave 'eval_f1' (o cualquier otra métrica de evaluación).
    if 'eval_f1' in log:
        # Comprobamos si el F1-score de la evaluación actual es superior al mejor F1-score
        # registrado hasta el momento.
        if log['eval_f1'] > best_metric:
            # Si es así, actualizamos `best_metric` con el nuevo valor.
            best_metric = log['eval_f1']
            # Extraemos el número de paso ('step') en el que se realizó esta evaluación.
            # Los checkpoints se guardan en directorios nombrados como `checkpoint-<step>`.
            step = log['step']
            # Construimos la ruta completa al directorio del mejor checkpoint utilizando
            # `training_args.output_dir` (el directorio base de resultados) y el número de paso.
            best_checkpoint_dir = os.path.join(training_args.output_dir, f"checkpoint-{step}")

# Después de revisar todo el historial, si `best_checkpoint_dir` no es None y existe físicamente,
# significa que se encontró un mejor checkpoint.
if best_checkpoint_dir and os.path.exists(best_checkpoint_dir):
    # Imprimimos la ruta del mejor checkpoint y su correspondiente F1-score.
    print(f"El mejor checkpoint es: {best_checkpoint_dir} con un F1-score de: {best_metric:.4f}")
    print("\nCargando el mejor modelo...")
    # Cargamos el modelo (sus pesos y configuración) desde el directorio del mejor checkpoint.
    # `AutoModelForSequenceClassification.from_pretrained()` es la forma estándar de hacer esto
    # para modelos de Transformers.
    best_model = AutoModelForSequenceClassification.from_pretrained(best_checkpoint_dir)
    print("¡Mejor modelo cargado y listo para usar!")

    # ==============================================================================
    # 5. EVALUACIÓN FINAL DEL MEJOR MODELO CARGADO
    # Una vez que el mejor modelo ha sido identificado y cargado, es una buena práctica
    # realizar una evaluación final explícita en el conjunto de prueba para confirmar
    # su rendimiento. Esto es especialmente útil si `load_best_model_at_end` no se usó.
    # ==============================================================================
    print("\nEvaluando el MEJOR modelo en el conjunto de prueba...")
    # Para la evaluación final, podemos crear un nuevo `Trainer` usando el `best_model`
    # que acabamos de cargar. Esto asegura que la evaluación se realice con el modelo óptimo.
    final_trainer = Trainer(model=best_model, # El modelo que hemos identificado como el mejor.
                            args=training_args, # Reutilizamos los argumentos de entrenamiento, ya que contienen
                                                # la configuración de evaluación (ej. batch size).
                            eval_dataset=tokenized_test_dataset, # El conjunto de datos de prueba para la evaluación final.
                            compute_metrics=compute_metrics) # La función de métricas para calcular los resultados.
    # Ejecutamos el método `evaluate()` del `final_trainer` para obtener las métricas
    # de rendimiento del mejor modelo en el conjunto de prueba.
    eval_results = final_trainer.evaluate()
    # Imprimimos los resultados de la evaluación final del mejor modelo.
    print(f"\nResultados de la evaluación del MEJOR modelo: {eval_results}")

else:
    # Si `best_checkpoint_dir` es `None` o el directorio no existe (lo cual sería inusual si el entrenamiento
    # ocurrió correctamente y se generaron logs de evaluación), significa que no se pudo identificar
    # un "mejor" checkpoint basándose en las métricas registradas. En este caso, se asume que
    # el modelo final que `trainer.train()` dejó es el que se debe evaluar.
    print("\nNo se encontraron checkpoints de evaluación con métricas válidas. Evaluando el último modelo entrenado.")
    # Se evalúa el modelo tal como quedó al finalizar el entrenamiento (`trainer.model`).
    # Aunque no sea el "mejor" según la métrica F1, es el resultado del entrenamiento completo.
    eval_results = trainer.evaluate()
    # La variable `best_model` apuntará al último estado del modelo entrenado.
    best_model = trainer.model
    print(f"Resultados de la evaluación final (último modelo): {eval_results}")

# A partir de este punto, la variable `best_model` contendrá el modelo de Transformer
# que se identificó como el mejor durante el entrenamiento (o el último entrenado si
# no se encontraron mejores checkpoints), listo para ser utilizado en tareas de inferencia
# (predicción de sentimiento para nuevas reseñas).


Configurando argumentos de entrenamiento...

Creando el Trainer...


/tmp/ipython-input-37-2464665707.py:115: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Iniciando el entrenamiento del modelo...


Step,Training Loss
100,0.674000
200,0.401800
300,0.326800
400,0.340500
500,0.345800
600,0.311500
700,0.312900
800,0.271400
900,0.284800
1000,0.270900



Entrenamiento completado.

Buscando el mejor checkpoint según la métrica 'f1'...

No se encontraron checkpoints de evaluación con métricas válidas. Evaluando el último modelo entrenado.


Resultados de la evaluación final (último modelo): {'eval_loss': 0.31700822710990906, 'eval_accuracy': 0.9289099526066351, 'eval_f1': 0.9291813159216474, 'eval_precision': 0.9290879871434311, 'eval_recall': 0.9292746634518786, 'eval_runtime': 148.8449, 'eval_samples_per_second': 66.626, 'eval_steps_per_second': 4.165, 'epoch': 3.0}


#**Bloque 8: Guardado del Modelo y División para GitHub**

Este bloque guardará el modelo en Google Drive y luego implementará la lógica para dividir el archivo pytorch_model.bin si es
demasiado grande.

**@title 8. Guardado del Modelo y División para GitHub**

In [ ]:
# 10. Guardar el Modelo en Google Drive
model_save_path_drive = os.path.join(DRIVE_BASE_PATH, "fine_tuned_sentiment_model_full_data")
print(f"\nGuardando el modelo y tokenizador en Google Drive: {model_save_path_drive}")
trainer.save_model(model_save_path_drive)
tokenizer.save_pretrained(model_save_path_drive)
print("Modelo y tokenizador guardados exitosamente en Google Drive.")

--- Lógica para dividir el modelo para GitHub ---
GitHub tiene un límite de 100MB por archivo.
El archivo principal del modelo suele ser 'pytorch_model.bin' o 'model.safetensors'.

In [ ]:
model_bin_path = os.path.join(model_save_path_drive, "pytorch_model.bin")
if not os.path.exists(model_bin_path):
    # Si no es pytorch_model.bin, podría ser safetensors
    model_bin_path = os.path.join(model_save_path_drive, "model.safetensors")

In [ ]:
if os.path.exists(model_bin_path):
    file_size_mb = os.path.getsize(model_bin_path) / (1024 * 1024)
    print(f"\nTamaño del archivo del modelo ({os.path.basename(model_bin_path)}): {file_size_mb:.2f} MB")

    if file_size_mb > 90: # Usamos 90MB como umbral para estar seguros por debajo de 100MB
        print(f"El archivo del modelo ({os.path.basename(model_bin_path)}) excede los 90MB. Dividiendo...")

        def split_file(filepath, chunk_size_mb=90):
            chunk_size = int(chunk_size_mb * 1024 * 1024)
            base_filename = os.path.basename(filepath)
            output_dir = os.path.dirname(filepath)

            with open(filepath, 'rb') as f:
                part_num = 0
                while True:
                    chunk = f.read(chunk_size)
                    if not chunk:
                        break
                    part_filename = os.path.join(output_dir, f"{base_filename}.part{part_num:03d}")
                    with open(part_filename, 'wb') as part_f:
                        part_f.write(chunk)
                    print(f"  Creada parte: {os.path.basename(part_filename)}")
                    part_num += 1
            print(f"Archivo '{base_filename}' dividido en {part_num} partes en {output_dir}.")
            print("Puedes subir estas partes a GitHub. Recuerda NO subir el archivo original grande.")
            print("Para reconstruir, usa el comando 'cat' o la función 'join_files' proporcionada.")

        split_file(model_bin_path)
    else:
        print("El archivo del modelo es menor de 90MB. No es necesario dividirlo para GitHub.")
else:
    print(f"Advertencia: No se encontró el archivo principal del modelo ({os.path.basename(model_bin_path)}).")

Instrucciones para el Usuario

1. Crea un nuevo Notebook en Google Colab:
    * Ve a Google Colab (https://colab.research.google.com/).
    * Haz clic en Archivo -> Nuevo notebook.
2. Copia y Pega los Bloques de Código:
    * Copia cada bloque de código que te he proporcionado y pégalo en celdas separadas en tu nuevo notebook.
    * Ejecuta cada celda en orden.
3. Asegúrate de que la carpeta `SLM_Training_Results` exista en tu Google Drive antes de ejecutar el notebook, o cámbiala a una
ruta que prefieras.
4. Conectar Colab con GitHub (para commits):
    * Una vez que el entrenamiento haya terminado y tengas el notebook con los resultados, puedes guardarlo en GitHub.
    * Ve a Archivo -> Guardar una copia en GitHub....
    * Sigue las instrucciones para autorizar Colab y seleccionar tu repositorio.
    * Para los archivos del modelo divididos, tendrás que descargarlos de Google Drive a tu máquina local y luego subirlos
    manualmente a GitHub (o usar git directamente en Colab si clonas tu repositorio y trabajas dentro de él, lo cual es más
    avanzado).

Función para Reconstruir el Modelo (para uso local)

Si necesitas reconstruir el modelo a partir de las partes descargadas de GitHub en tu máquina local, usa esta función Python:

In [ ]:
import os

In [ ]:
def join_files(output_filepath, part_prefix):
    """
    Reconstruye un archivo a partir de sus partes.
    output_filepath: Ruta completa del archivo final a reconstruir.
    part_prefix: Prefijo de los archivos de las partes (ej.
    "./fine_tuned_sentiment_model_full_data/pytorch_model.bin.part").
                 Asegúrate de que incluya la ruta completa a las partes.
    """
    print(f"Reconstruyendo archivo en: {output_filepath}")
    with open(output_filepath, 'wb') as outfile:
        part_num = 0
        while True:
            # Formato de nombre de parte: .part000, .part001, etc.
            part_filename = f"{part_prefix}{part_num:03d}"
            if not os.path.exists(part_filename):
                break
            print(f"  Añadiendo parte: {os.path.basename(part_filename)}")
            with open(part_filename, 'rb') as infile:
                outfile.write(infile.read())
            part_num += 1
    if part_num > 0:
        print(f"Archivo reconstruido exitosamente en '{output_filepath}' a partir de {part_num} partes.")
    else:
        print(f"Advertencia: No se encontraron partes con el prefijo '{part_prefix}'.")

Ejemplo de uso (ajusta las rutas según donde descargues las partes):

- model_dir = "./fine_tuned_sentiment_model_full_data" # Carpeta donde están las partes
- output_file = os.path.join(model_dir, "pytorch_model.bin")
- part_base_name = "pytorch_model.bin.part" # Nombre base del archivo original
- join_files(output_file, os.path.join(model_dir, part_base_name))